In [ ]:
import os

import numpy as np
import pandas as pd

from covidvu import utils
from covidvu.cryostation import Cryostation

In [ ]:
WINDOW_SIZE = 7
MIN_CASES = 100

In [ ]:
databasePath = './database/virustrack.db'
storage = Cryostation(databasePath=databasePath)
countries = storage.timeSeriesFor(regionType = 'country', casesType = 'confirmed', disableProgressBar=False)

In [ ]:
statesUS = storage.timeSeriesFor(regionType = 'province', countryName='US', 
                                 casesType = 'confirmed', disableProgressBar=False)

In [ ]:
def computeGrowthFactor(confirmedCases, windowSize):    
    confirmedCasesSmooth = confirmedCases.copy().rolling(windowSize, axis=0).mean()
    newCases = confirmedCasesSmooth.diff(axis=0)
    growthFactor = newCases/newCases.shift(1)
    return growthFactor

In [ ]:
growthCountries = computeGrowthFactor(countries, WINDOW_SIZE)

In [ ]:
growthStatesUS = computeGrowthFactor(statesUS, WINDOW_SIZE)

In [ ]:
type(growthStatesUS.index)

In [ ]:
todayDate = pd.Timestamp.today().date()
yesterdayDate = (pd.Timestamp.today() - pd.Timedelta(1,'D')).date()
lastWeekDate = (pd.Timestamp.today() - pd.Timedelta(7,'D')).date()
lastTwoWeekDate = (pd.Timestamp.today() - pd.Timedelta(14,'D')).date()

In [ ]:
str(todayDate)

In [ ]:
def getGrowthGaugeData(growth):
    try:
        mostRecentGrowth = growth.loc[todayDate]
        mostRecentDate   = todayDate
    except KeyError:
        mostRecentGrowth = growth.loc[yesterdayDate]
        mostRecentDate   = yesterdayDate
        
    lastWeekGrowth = growth.loc[lastWeekDate]
    lastTwoWeekGrowth = growth.loc[lastTwoWeekDate]
    return {
                str(mostRecentDate): mostRecentGrowth, 
                str(lastWeekDate): lastWeekGrowth, 
                str(lastTwoWeekDate): lastTwoWeekGrowth,
           }

In [ ]:
growthGaugeDataCountries = getGrowthGaugeData(growthCountries)
growthGaugeDataStatesUS  = getGrowthGaugeData(growthStatesUS)

In [ ]:
growthGaugeDataCountries['2020-04-13']['US']

In [ ]:
growthGaugeDataCountries.keys()

In [ ]:
{d:growthGaugeDataCountries[d]['US'] for d in growthGaugeDataCountries.keys()}

In [ ]:
from covidvu.config import MASTER_DATABASE as database

In [ ]:
with Cryostation(database) as cryostation:
    cryostation['US']['growth'] = {d:growthGaugeDataCountries[d]['US'] for d in growthGaugeDataCountries.keys()}